In [1]:
import matplotlib.pyplot as plt
from spacepy import pycdf
import numpy as np
import matplotlib.dates as mdates
import math
import datetime as dt
from datetime import datetime
from matplotlib.axis import Axis
from matplotlib.pyplot import cm
from itertools import compress

class rpw_hfr_lab:
    
    def __init__(self, date, start_event, end_event, filepath):
        self.date = date
        self.start_event = start_event
        self.end_event = end_event
        self.filepath = filepath
    
    @staticmethod
    def array_dupes(a):
        u, c = np.unique(a, return_counts=True)
        dup = u[c > 1]
        return len(dup)
    @staticmethod
    def to_timestamp(array): # Converts datetime arrays into timestamp arrays
        timestamps = list(map(lambda x: datetime.timestamp(x), array))
        return timestamps
    @staticmethod
    def to_date(array): # Converts timestamp arrays into datetime arrays
        mydates = list(map(lambda x: datetime.fromtimestamp(x), array))
        return mydates
    @staticmethod
    def freq_chooser(a, m):
        choice = list(map(lambda x: len(x) == m, a))
        return choice
    @staticmethod
    def available_time(t, dt):
        return np.mod(t, dt)
    @staticmethod
    def join_sensors(x, y, n):
        x_non = np.where(x <= n)
        y_non = np.where(y > n)
        s = x_non[0]
        x[x_non] = y[y_non]
        return x
    
    def header(self):
        head_txt = """
        Import the following modules:
        
        import matplotlib.pyplot as plt
        from spacepy import pycdf
        import numpy as np
        import matplotlib.dates as mdates
        import math
        import datetime as dt
        from datetime import datetime
        from matplotlib.axis import Axis
        from matplotlib.pyplot import cm
        from itertools import compress
        
        Example:
        my_data = rpw_hfr_lab('2021-10-28', '00:00:00', '20:00:00', 'RPW_plotting\\solo_L2_rpw-hfr-surv-cdag_20211028_V05')
        
        my_data.data()['frequencies']
        >>> frequency array in kHz
        my_data.data()['time']
        >>> time array
        my_data.data()['l2_spectrum']
        >>> dynamic spectrum in V^2/Hz
        
        my_data.plot("hfr_dynamic_spectrum\\non_cal\\")
        >>> plots the dynamic spectrum in dB and saves it as 'hfr_dynspec_noncal_date.png' in the path you indicate
        """
        print(head_txt)
    
    def data(self):
        
        cdf_file = pycdf.CDF(self.filepath)
        # Open variables
        length = len(cdf_file['FREQUENCY'])

        freq, epoch, AGC, sensor = cdf_file['FREQUENCY'][...], cdf_file['Epoch'][...], self.join_sensors(cdf_file['AGC1'][...], cdf_file['AGC2'][...], 0), self.join_sensors(cdf_file['SENSOR_CONFIG'][0:length,0], cdf_file['SENSOR_CONFIG'][0:length,1], 8)
        # Check for what indexes each frequency appears 
        freq_num = self.array_dupes(freq)
        freq_list = freq[0:freq_num]
        freq_choice_ = list(map(lambda x: list(np.array(np.where(freq==x)[0])), freq_list))
        # Determine which frequencies to use in function of the length of the index arrays
        max_len = np.max(list(map(lambda x: len(x), freq_choice_)))
        freq_choice = list(compress(freq_choice_,self.freq_chooser(freq_choice_, max_len)))
        frequencies = list(compress(freq_list, self.freq_chooser(freq_choice_, max_len)))
        freq_num = len(frequencies)
        # For each frequency, apply same indexes to time and flux
        epoch_choice = epoch[freq_choice]
        AGC_choice = AGC[freq_choice]

        yyyy, mm, dd = self.date.split('-')
        HH_s, MM_s, SS_s = self.start_event.split(':')
        HH_e, MM_e, SS_e = self.end_event.split(':')
        time_start = dt.datetime(int(yyyy), int(mm), int(dd), int(HH_s), int(MM_s), int(SS_s))
        time_end = dt.datetime(int(yyyy), int(mm), int(dd), int(HH_e), int(MM_e), int(SS_e))

        def time_inter(t_arr):
            start = np.abs(np.array(self.to_timestamp(t_arr))-datetime.timestamp(time_start)).argmin()
            end = np.abs(np.array(self.to_timestamp(t_arr))-datetime.timestamp(time_end)).argmin()
            return [start, end]

        middle_freq = math.ceil(len(frequencies)/2.)
        start = time_inter(epoch_choice[middle_freq])[0]
        end = time_inter(epoch_choice[middle_freq])[1]

        epoch_choice_plot = epoch_choice[:, start:end]
        AGC_choice_plot = AGC_choice[:, start:end]
        middle_freq = math.ceil(len(frequencies)/2.)

        Spectro = []

        for i in list(range(0, freq_num)):
            order = np.argsort(self.to_timestamp(epoch_choice_plot[i]))
            AGC_specter = np.interp(np.sort(self.to_timestamp(epoch_choice_plot[middle_freq])), np.sort(self.to_timestamp(epoch_choice_plot[i])), AGC_choice_plot[i][order]) - np.median(AGC_choice_plot[i])
            #AGC1_specter = np.interp(np.sort(to_timestamp(epoch_choice_plot[middle_freq])), to_timestamp(epoch_choice_plot[i]), AGC_choice_plot[i]) - np.median(AGC_choice_plot[i])
            AGC_specter[AGC_specter <= 0] = np.median(AGC_choice_plot[i])
            Spectro.append(list(AGC_specter))
        data_dict = {'time': self.to_date(np.sort(self.to_timestamp(epoch_choice_plot[middle_freq]))), 'frequencies': np.array(frequencies), 'l2_spectrum': Spectro}

        return data_dict
    
    def plot(self, fig_path):
        
        dic_to_plot = rpw_hfr_lab(self.date, self.start_event, self.end_event, self.filepath).data()
        time_arr, freq_arr, Spectro = dic_to_plot['time'], dic_to_plot['frequencies'], dic_to_plot['l2_spectrum']
        
        myFmt = mdates.DateFormatter('%H:%M')
        fig = plt.figure(figsize=(15,7))
        ax_spectro = fig.add_subplot(111)
        Spectrum = 10*np.log10(Spectro)
        #v_min_val, v_max_val =np.log10(np.min(Spectro)) + 10, 10*np.log10((np.max(Spectro) + np.min(Spectro))/2)
        v_min_val, v_max_val = np.min(Spectrum) + 50, 10*np.log10((np.max(Spectro) + np.min(Spectro))/2.)
        T,F = np.meshgrid(time_arr, freq_arr)
        #im = ax_spectro.pcolormesh(T, F, Spectro, cmap=cm.inferno, vmin=v_min_val , vmax=v_max_val)
        im = ax_spectro.pcolormesh(T, F, Spectrum, cmap=cm.inferno, vmin=v_min_val , vmax=v_max_val)
        cbar = fig.colorbar(im)
        cbar.ax.tick_params(labelsize=15)
        cbar.set_label('dB', fontsize=20)
        ax_spectro.xaxis.set_major_formatter(myFmt)
        ax_spectro.tick_params(axis='both', which='both', labelsize=15, length=5, width=1)
        ax_spectro.set_yscale("log")
        ax_spectro.set_xlabel('Time (HH:MM)', fontsize=20)
        ax_spectro.set_ylabel('Frequency (kHz)', fontsize=20)
        ax_spectro.set_title(self.date + ' (yyyy-mm-dd)', fontsize=20)
        plt.savefig(fig_path + "hfr_dynspec_noncal_" + self.date + ".png", bbox_inches='tight')

C:\Users\franc\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\franc\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\franc\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
#result = rpw_hfr_lab('2021-10-28', '00:00:00', '20:00:00', 'RPW_plotting\\solo_L2_rpw-hfr-surv-cdag_20211028_V05')

In [3]:
#result.header()

In [4]:
#result.data()['time']

In [5]:
#result.plot("hfr_dynamic_spectrum\\non_cal\\")